In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import awkward as ak
import uproot 
import h5py
import json
import copy
from pprint import pprint
import time
from rich.progress import track
import os,sys,glob,re,gc
import numpy as np
import matplotlib.pyplot as plt

### Load dijet and Wtop sample dictionaries for T3

In [3]:
with open('Unfolding_DijetDict_NominalV9.json') as dictio:
    t3_samples_dijets = json.load(dictio)
with open('Unfolding_WtopDict_NominalV4.json') as dictio:
    t3_samples_Wtop = json.load(dictio)

t3_samples_dijets_copy=copy.deepcopy(t3_samples_dijets)
t3_samples_Wtop_copy=copy.deepcopy(t3_samples_Wtop)

In [4]:
for sn in t3_samples_dijets_copy.keys():
    if not('H7' in sn or sn.startswith('QCD_HT')):
        del(t3_samples_dijets[sn])
for sn in t3_samples_Wtop_copy.keys():
    if not(sn.startswith('TTToSemiLeptonic')):
        del(t3_samples_Wtop[sn])
del(t3_samples_dijets_copy)        
del(t3_samples_Wtop_copy)        
#pprint(t3_samples_Wtop)
#pprint(t3_samples_dijets)


In [5]:
#files stored in dir+maskType+year/
dir_dijets = '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/'
dir_Wtop = '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Wtop_rootToParquet/'
maskTypes = ['recoMasked/','genMasked/']
year_list = ['2017','2018','2016_preVFP','2016']
outputDir = '/work/kadatta/private/CMSSW_10_6_29/src/nSubHistogramming/V9/ML/'

In [6]:
#using only reco and gen for now (not true-/fake-reco or accep-/miss-gen)
for mask in maskTypes:
    for year in year_list:
        inpDir = dir_Wtop+mask+year+'/'
        file_list = os.listdir(inpDir)
        for sn in t3_samples_Wtop.keys():
            
            temp = [i for i in file_list if i.startswith(sn) and (year in i)]
            #print(year,sn,temp)
            for i in temp:
                  for sel in ['WSel','topSel']:
                    if 'reco' in mask:
                        if 'true' in i or 'fake' in i or not(sel in i): continue
                        #print(i)
                        t3_samples_Wtop[sn][year][f'parquet_reco_{sel}'] = inpDir+i
                    elif 'gen'in mask:
                        if 'accep' in i or 'miss' in i or not(sel in i): continue
                        t3_samples_Wtop[sn][year][f'parquet_gen_{sel}'] = inpDir+i
                    
pprint(t3_samples_Wtop)                    

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': {'2016': {'MCScaling': 0.0001403967742183984,
                                                            'miniAOD': '/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v1/MINIAODSIM',
                                                            'nEvents': 144974000,
                                                            'nGenWeights': 43624104146.171936,
                                                            'nanoAOD': '/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/spigazzi-RunIISummer20UL16PFNanov2pt2-106X_mcRun2_asymptotic_v17-v1-36773dbd6a9d41d00afbee31f82bc069/USER',
                                                            'nevents': 1.0,
                                                            'parquet_gen_WSel': '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Wtop_rootToParquet/genMasked/2016/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_UL2016_nomWts_W

In [7]:
#using only reco and gen for now (not true-/fake-reco or accep-/miss-gen)
for mask in maskTypes:
    for year in year_list:
        inpDir = dir_dijets+mask+year+'/'
        file_list = os.listdir(inpDir)
        for sn in t3_samples_dijets.keys():
            if 'MLM' in sn:
                s=sn.split('MLM')[1]
            else: s=sn
            temp = [i for i in file_list if i.startswith(s) and (year in i)]
            #print(year,sn,temp)
            for i in temp:
                if 'reco' in mask:
                    if 'true' in i or 'fake' in i: continue
                    #print(i)
                    t3_samples_dijets[sn][year]['parquet_reco'] = inpDir+i
                elif 'gen'in mask:
                    if 'accep' in i or 'miss' in i: continue
                    t3_samples_dijets[sn][year]['parquet_gen'] = inpDir+i
pprint(t3_samples_dijets)

{'H7MLMQCD_HT1000to1500_TuneCH3_13TeV-madgraphMLM-herwig7': {'2016': {'MCScaling': 0.013375034075141826,
                                                                      'miniAOD': '/QCD_HT1000to1500_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL16MiniAODv2-106X_mcRun2_asymptotic_v17-v2/MINIAODSIM',
                                                                      'nEvents': 987782,
                                                                      'nGenWeights': 987782.002085098,
                                                                      'nanoAOD': '/QCD_HT1000to1500_TuneCH3_13TeV-madgraphMLM-herwig7/kadatta-RunIISummer20UL16PFNanoV4-106X_mcRun2_asymptotic_v17-v2-c5cba6a427fbfe3f4bbf74218753ca9a/USER',
                                                                      'parquet_gen': '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/genMasked/2016/QCD_HT1000to1500_TuneCH3_13TeV-madgraphMLM-herwig7_UL2016_nomWts_CentralJet_gen_

                                                                      'parquet_gen': '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/genMasked/2016/QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8_UL2016_nomWts_CentralJet_gen_0.parquet',
                                                                      'parquet_reco': '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/recoMasked/2016/QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8_UL2016_nomWts_CentralJet_reco_0.parquet',
                                                                      'skimmer': '/QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8/spigazzi-RunIISummer20UL16PFNanov2pt2-106X_mcRun2_asymptotic_v17-v1-36773dbd6a9d41d00afbee31f82bc069/USER',
                                                                      'skimmerHisto': 'jetObservables_histograms_QCDHT200to300_UL2016_nomWts.root',
                                              

### Listing branches that need to be kept

In [8]:
#'FlagGenLeptHemBjet_nom', 'btagWeightNom_nom', 'evtGenWeight_nom', 'fsrWeightDown_nom', 'fsrWeightUp_nom', 'good_nPVs_nom', 
#'isrWeightDown_nom', 'isrWeightUp_nom', 'l1prefiringWeightNom_nom', 'leptonWeightNom_nom', 'nGenBtags_nom', 
#'pdfWeightAll_nom', 'pdfWeightDown_nom', 'pdfWeightNom_nom', 'pdfWeightUp_nom', 'puWeightDown_nom', 'puWeightNom_nom', 
#'puWeightUp_nom', 
#'selGenLeptW_nom_eta', 'selGenLeptW_nom_mass', 'selGenLeptW_nom_phi', 'selGenLeptW_nom_pt', 'selGenLeptW_nom_y', 
#'selGenMu_nom_eta', 'selGenMu_nom_mass', 'selGenMu_nom_phi', 'selGenMu_nom_pt', 'selGenMu_nom_y'
#'selRecoAK4bjetHadHem_nom_pt', 'selRecoAK4bjetHadHem_nom_y','selRecoLeptW_nom_eta', 'selRecoLeptW_nom_mass',
#'selRecoLeptW_nom_phi', 'selRecoLeptW_nom_pt', 'selRecoLeptW_nom_y',
#'selRecoMu_nom_eta', 'selRecoMu_nom_mass', 'selRecoMu_nom_phi', 'selRecoMu_nom_pt', 'selRecoMu_nom_y', 

fields_Wtop_gen = [ 'selGenJets_nom_pt', 'selGenJets_nom_msoftdrop',  
                    'selGenJets_nom_tau_0p5_1', 'selGenJets_nom_tau_1_1', 'selGenJets_nom_tau_2_1',
                    'selGenJets_nom_tau_0p5_2', 'selGenJets_nom_tau_1_2', 'selGenJets_nom_tau_2_2',
                    'selGenJets_nom_tau_0p5_3', 'selGenJets_nom_tau_1_3', 'selGenJets_nom_tau_2_3',
                    'selGenJets_nom_tau_0p5_4', 'selGenJets_nom_tau_1_4', 'selGenJets_nom_tau_2_4', 
                    'selGenJets_nom_tau_0p5_5', 'selGenJets_nom_tau_1_5', 'selGenJets_nom_tau_2_5',
                    'selGenJets_nom_tau21', 'selGenJets_nom_tau32', 
                    'selGenJets_nom_tau21_WTA','selGenJets_nom_tau32_WTA', 
                    'selGenJets_nom_tau21_exkT','selGenJets_nom_tau32_exkT', 
                    'selGenHadHemDeltaR_nom', 'selGenJets_nom_phi', 'selGenJets_nom_eta', 
                    'selGenJets_nom_y', 'selGenJets_nom_mass',
                    'evtGenWeight_nom'
                  ]
fields_Wtop_reco = ['selRecoJets_nom_pt', 'selRecoJets_nom_msoftdrop', 
                    'selRecoJets_nom_tau_0p5_1', 'selRecoJets_nom_tau_1_1', 'selRecoJets_nom_tau_2_1',
                    'selRecoJets_nom_tau_0p5_2', 'selRecoJets_nom_tau_1_2', 'selRecoJets_nom_tau_2_2',
                    'selRecoJets_nom_tau_0p5_3', 'selRecoJets_nom_tau_1_3', 'selRecoJets_nom_tau_2_3',
                    'selRecoJets_nom_tau_0p5_4', 'selRecoJets_nom_tau_1_4', 'selRecoJets_nom_tau_2_4', 
                    'selRecoJets_nom_tau_0p5_5', 'selRecoJets_nom_tau_1_5', 'selRecoJets_nom_tau_2_5',
                    'selRecoJets_nom_tau21', 'selRecoJets_nom_tau32',
                    'selRecoJets_nom_tau21_WTA', 'selRecoJets_nom_tau32_WTA', 
                    'selRecoJets_nom_tau21_exkT', 'selRecoJets_nom_tau32_exkT', 
                    'selRecoHadHemDeltaR_nom', 'selRecoJets_nom_phi', 'selRecoJets_nom_eta', 
                    'selRecoJets_nom_y','selRecoJets_nom_mass',
                    'evtGenWeight_nom','totalRecoWeight_nom'
                    #'puWeightNom_nom','btagWeightNom_nom','leptonWeightNom_nom','l1prefiringWeightNom_nom',
                  ]

### Prep W/top datasets

In [9]:
fields_Wtop_gen.pop(fields_Wtop_gen.index('selGenHadHemDeltaR_nom'))
fields_Wtop_reco.pop(fields_Wtop_reco.index('selRecoHadHemDeltaR_nom'))

'selRecoHadHemDeltaR_nom'

In [10]:
for sn in t3_samples_Wtop.keys():
    for year in year_list:
        for sel in ['WSel','topSel']:
            gen_parquet_path = t3_samples_Wtop[sn][year][f'parquet_gen_{sel}']
            reco_parquet_path = t3_samples_Wtop[sn][year][f'parquet_reco_{sel}']
            genEvents = ak.from_parquet(gen_parquet_path,columns=fields_Wtop_gen) 
            recoEvents = ak.from_parquet(reco_parquet_path,columns=fields_Wtop_reco) 
            
            recoWeight = abs(recoEvents['totalRecoWeight_nom'])
            genWeight_reco = abs(recoEvents['evtGenWeight_nom'])
            genWeight_gen = abs(genEvents['evtGenWeight_nom'])
            MCScale = t3_samples_Wtop[sn][year]['MCScaling']
            
            if not(os.path.exists(outputDir+sel)): 
                os.makedirs(outputDir+sel)
                print(sel,year,h5_out_path)
            
            h5_out_path = outputDir+sel

            recoSave = np.array((recoEvents.to_numpy()).tolist())
            genSave = np.array((genEvents.to_numpy()).tolist())
            print(year,sel,len(genSave),len(recoSave))
            
            with h5py.File(h5_out_path+f'/ttbar_{sel}_{year}_reco.h5','w') as f:
                f.create_dataset('inputs',data=recoSave)
                f.create_dataset('target',data=np.ones(len(recoSave)))
                f.create_dataset('genWeights',data=np.array(list(genWeight_reco)))
                f.create_dataset('recoWeights',data=np.array(list(recoWeight)))
                f.create_dataset('MCScale',data=np.ones(len(recoSave))*MCScale)
                f.close()

            with h5py.File(h5_out_path+f'/ttbar_{sel}_{year}_gen.h5','w') as f:
                f.create_dataset('inputs',data=genSave)
                f.create_dataset('target',data=np.ones(len(genSave)))
                f.create_dataset('genWeights',data=np.array(list(genWeight_gen)))
                f.create_dataset('MCScale',data=np.ones(len(genSave))*MCScale)
                f.close()
gc.collect()

2017 WSel 123340 95670
2017 topSel 83760 77420
2018 WSel 168678 128994
2018 topSel 113549 100196
2016_preVFP WSel 46202 32874
2016_preVFP topSel 31627 26604
2016 WSel 51073 37498
2016 topSel 34536 29636


0

### Prep QCD/bkg datasets

In [27]:
#bkg_MC_total_nevents = 0.
#bkg_MC_norm_list = []
#bkg_norm_dict = dict()
#for sn in t3_samples_dijets.keys():
#    if not(sn.startswith('QCD_HT')):
#        continue
#    print(sn)
#    bkg_MC_total_nevents+=t3_samples_dijets[sn]['2018']['nGenWeights']
#    bkg_MC_norm_list.append(t3_samples_dijets[sn]['2018']['nGenWeights'])
#print(bkg_MC_total_nevents,bkg_MC_norm_list)
#bkg_MC_norm_list=[i/bkg_MC_total_nevents for i in bkg_MC_norm_list]
#print(bkg_MC_total_nevents,bkg_MC_norm_list)


QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT300to500_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT500to700_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT700to1000_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT1500to2000_TuneCP5_PSWeights_13TeV-madgraph-pythia8
QCD_HT2000toInf_TuneCP5_PSWeights_13TeV-madgraph-pythia8
331591046.0 [84083628.0, 57336623.0, 61705174.0, 49184771.0, 48506751.0, 14527915.0, 10871473.0, 5374711.0]
331591046.0 [0.2535762922862519, 0.1729136648641592, 0.18608817923268048, 0.1483296114093503, 0.14628486379574918, 0.04381274818862268, 0.03278578577782224, 0.016208854445364004]


In [16]:
sel='dijetSel'

for selLike in ['WSel','topSel']:
    
    for sn in t3_samples_dijets.keys():
        if not(sn.startswith('QCD_HT')): continue #if not('H7' in sn): continue

        for year in year_list:

            gen_parquet_path = t3_samples_dijets[sn][year][f'parquet_gen']
            reco_parquet_path = t3_samples_dijets[sn][year][f'parquet_reco']
            
            #mask qcd to get W-/top-like events
            genEvents = ak.from_parquet(gen_parquet_path,columns=fields_Wtop_gen) 
            recoEvents = ak.from_parquet(reco_parquet_path,columns=fields_Wtop_reco) 
            try:
                if 'W' in selLike:
                    genMask = (genEvents.evtGenWeight_nom!=0.) & (genEvents.selGenJets_nom_pt>200.) & (genEvents.selGenJets_nom_msoftdrop>65.) & (genEvents.selGenJets_nom_msoftdrop<120.) 
                    recoMask = (recoEvents.totalRecoWeight_nom!=0.) & (recoEvents.selRecoJets_nom_pt>200.) & (recoEvents.selRecoJets_nom_msoftdrop>65.) & (recoEvents.selRecoJets_nom_msoftdrop<120.) 
                elif 'top' in selLike:
                    genMask = (genEvents.evtGenWeight_nom!=0.) & (genEvents.selGenJets_nom_pt>400.) & (genEvents.selGenJets_nom_msoftdrop>140.) & (genEvents.selGenJets_nom_msoftdrop<250.) 
                    recoMask = (recoEvents.totalRecoWeight_nom!=0.) & (recoEvents.selRecoJets_nom_pt>400.) & (recoEvents.selRecoJets_nom_msoftdrop>140.) & (recoEvents.selRecoJets_nom_msoftdrop<250.) 

                print(year,sel,len(genEvents),len(recoEvents))

                genEvents = genEvents[genMask]
                recoEvents = recoEvents[recoMask]
                
                recoWeight = abs(recoEvents['totalRecoWeight_nom'])
                genWeight_reco = abs(recoEvents['evtGenWeight_nom'])
                genWeight_gen = abs(genEvents['evtGenWeight_nom'])
                MCScale = t3_samples_dijets[sn][year]['MCScaling']
                
                if not(os.path.exists(outputDir+sel)): 
                    os.makedirs(outputDir+sel)
                    h5_out_path = outputDir+sel

                    print(sel,year,h5_out_path)

                h5_out_path = outputDir+sel
                pref = sn.split('_Tune')[0]#'_H7' if ('herwig' in sn.lower()) else ''
                recoSave = np.array((recoEvents.to_numpy()).tolist())
                genSave = np.array((genEvents.to_numpy()).tolist())
                print(year,selLike,len(genSave),len(recoSave))

                with h5py.File(h5_out_path+f'/{pref}_{selLike}_{year}_reco.h5','w') as f:
                    f.create_dataset('inputs',data=recoSave)                   
                    f.create_dataset('genWeights',data=np.array(list(genWeight_reco)))
                    f.create_dataset('recoWeights',data=np.array(list(recoWeight)))
                    f.create_dataset('MCScale',data=np.ones(len(recoSave))*MCScale)
                    f.create_dataset('target',data=np.zeros(len(recoSave)))
                    f.close()

                with h5py.File(h5_out_path+f'/{pref}_{selLike}_{year}_gen.h5','w') as f:
                    f.create_dataset('inputs',data=genSave)
                    f.create_dataset('genWeights',data=np.array(list(genWeight_gen)))
                    f.create_dataset('MCScale',data=np.ones(len(genSave))*MCScale)
                    f.create_dataset('target',data=np.zeros(len(genSave)))
                    f.close()
            except KeyError:
                if selLike=='WSel':
                    print(f"Passing on {sn} in {year}")
                    print(gen_parquet_path)
                    print(reco_parquet_path)
                
gc.collect()

2017 dijetSel 1 654
2017 WSel 0 2
2017 dijetSel 1 654
2017 topSel 0 0
2018 dijetSel 1 810
2018 WSel 0 0
2018 dijetSel 1 810
2018 topSel 0 0
Passing on QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8 in 2016_preVFP
/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/genMasked/2016_preVFP/QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8_UL2016_preVFP_nomWts_CentralJet_gen_0.parquet
/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/recoMasked/2016_preVFP/QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8_UL2016_preVFP_nomWts_CentralJet_reco_0.parquet
Passing on QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8 in 2016
/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/genMasked/2016/QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8_UL2016_nomWts_CentralJet_gen_0.parquet
/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/Dijets_rootToParquet/recoMasked/2016/QCD_HT1

887

### Check, pre-process, and shuffle signal/background datasets

In [8]:
sig_norm_dict_reco = dict()
sig_reco_sum=0.
sig_norm_dict_gen = dict()
sig_gen_sum=0.

for sel in ['WSel','topSel']:
    sig_reco_sum=0.
    sig_gen_sum=0.

    #for 'ttbar_'+sel in t3_samples_Wtop.keys():

    sig_norm_dict_reco['ttbar_'+sel] = dict()
    sig_norm_dict_gen['ttbar_'+sel] = dict()

    for year in year_list:
        sig_norm_dict_reco['ttbar_'+sel][year] = dict()
        sig_norm_dict_gen['ttbar_'+sel][year] = dict()

        h5_in_path = outputDir+sel
        try: 
            with h5py.File(h5_in_path+f'/ttbar_{sel}_{year}_reco.h5','r') as freco:
                sig_norm_dict_reco['ttbar_'+sel][year]['n_events'] = len(freco['inputs'])
                freco.close()
        except OSError:
            sig_norm_dict_reco['ttbar_'+sel][year]['n_events'] = 0

        try:   
            with h5py.File(h5_in_path+f'/ttbar_{sel}_{year}_gen.h5','r') as fgen:
                sig_norm_dict_gen['ttbar_'+sel][year]['n_events'] = len(fgen['inputs'])
                fgen.close()
        except OSError:
            sig_norm_dict_gen['ttbar_'+sel][year]['n_events'] = 0

        sig_reco_sum+=sig_norm_dict_reco['ttbar_'+sel][year]['n_events']
        sig_gen_sum+=sig_norm_dict_gen['ttbar_'+sel][year]['n_events'] 
    
    sig_norm_dict_reco['ttbar_'+sel]['totalEvents']=sig_reco_sum
    sig_norm_dict_gen['ttbar_'+sel]['totalEvents']=sig_gen_sum
    
    print(sel,sig_gen_sum,sig_reco_sum)

WSel 389293.0 295036.0
topSel 263472.0 233856.0


In [9]:
pprint(sig_norm_dict_gen)
pprint(sig_norm_dict_reco)

{'ttbar_WSel': {'2016': {'n_events': 51073},
                '2016_preVFP': {'n_events': 46202},
                '2017': {'n_events': 123340},
                '2018': {'n_events': 168678},
                'totalEvents': 389293.0},
 'ttbar_topSel': {'2016': {'n_events': 34536},
                  '2016_preVFP': {'n_events': 31627},
                  '2017': {'n_events': 83760},
                  '2018': {'n_events': 113549},
                  'totalEvents': 263472.0}}
{'ttbar_WSel': {'2016': {'n_events': 37498},
                '2016_preVFP': {'n_events': 32874},
                '2017': {'n_events': 95670},
                '2018': {'n_events': 128994},
                'totalEvents': 295036.0},
 'ttbar_topSel': {'2016': {'n_events': 29636},
                  '2016_preVFP': {'n_events': 26604},
                  '2017': {'n_events': 77420},
                  '2018': {'n_events': 100196},
                  'totalEvents': 233856.0}}


In [10]:
bkg_norm_dict_reco = dict()
bkg_norm_dict_gen = dict()
sel='dijetSel'


for selLike in [['WSel','Wlike'],['topSel','toplike']]:
    bkg_reco_sum=0.
    bkg_gen_sum=0.
    bkg_norm_dict_reco['totalEvents'+'_'+selLike[0]] = dict()
    bkg_norm_dict_gen['totalEvents'+'_'+selLike[0]] = dict()
    for y,year in enumerate(year_list):
        XS = 0.
        bkg_reco_sum_peryear = 0.
        bkg_gen_sum_peryear = 0.
        for sn in t3_samples_dijets.keys():
            if not(sn.startswith('QCD_HT')): continue #if not('H7' in sn): continue
            if y==0:
                bkg_norm_dict_reco[sn+'_'+selLike[0]] = dict()
                bkg_norm_dict_gen[sn+'_'+selLike[0]] = dict()
            
            bkg_norm_dict_reco[sn+'_'+selLike[0]][year] = dict()
            bkg_norm_dict_gen[sn+'_'+selLike[0]][year] = dict()
        
            pref = sn.split('_Tune')[0]#'_H7' if ('herwig' in sn.lower()) else ''
            h5_in_path = outputDir+sel
            try: 
                with h5py.File(h5_in_path+f'/{selLike[1]}/{pref}_{selLike[0]}_{year}_reco.h5','r') as freco:
                    bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events'] = len(freco['inputs'])
                    freco.close()
            except OSError:
                bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events'] = 0
            
            try:   
                with h5py.File(h5_in_path+f'/{selLike[1]}/{pref}_{selLike[0]}_{year}_gen.h5','r') as fgen:
                    bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events'] = len(fgen['inputs'])
                    fgen.close()
            except OSError:
                bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events'] = 0
            
            bkg_reco_sum+=bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events']
            bkg_gen_sum+=bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events'] 
            
            bkg_reco_sum_peryear+=bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events']
            bkg_gen_sum_peryear+=bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events'] 
            
            XS+=t3_samples_dijets[sn]['XS']
        
        
        bkg_norm_dict_reco['totalEvents'+'_'+selLike[0]][year] = bkg_reco_sum_peryear
        bkg_norm_dict_gen['totalEvents'+'_'+selLike[0]][year] = bkg_gen_sum_peryear
        
        bkg_norm_dict_gen['sum_XS'] = XS
        bkg_norm_dict_reco['sum_XS'] = XS
        
print(sig_reco_sum,sig_gen_sum)
print(bkg_reco_sum,bkg_gen_sum)
print(sig_reco_sum/bkg_reco_sum,sig_gen_sum/bkg_gen_sum)

#pprint(bkg_norm_dict_gen)
#pprint(bkg_norm_dict_reco)


for selLike in [['WSel','Wlike'],['topSel','toplike']]:

    for year in year_list:
        
        for sn in t3_samples_dijets.keys():

            if not(sn.startswith('QCD_HT')): continue #if not('H7' in sn): continue

            #rc = 10**10 if bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events']==0 else bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events']
            #gc = 10**10 if bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events']==0 else bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events']
            
            #bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['norm'] = ( t3_samples_dijets[sn]['XS']/bkg_norm_dict_reco['sum_XS'])

            #bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['norm'] =  ( t3_samples_dijets[sn]['XS']/bkg_norm_dict_gen['sum_XS'])

            bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['keep_events_nEvt'] = int( bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events']/bkg_norm_dict_reco['totalEvents'+'_'+selLike[0]][year]*sig_norm_dict_reco['ttbar_'+selLike[0]][year]['n_events'] )

            bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['keep_events_nEvt'] =  int( bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events']/bkg_norm_dict_gen['totalEvents'+'_'+selLike[0]][year]*sig_norm_dict_gen['ttbar_'+selLike[0]][year]['n_events'] )

            #bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['keep_events_nEvtXS'] = int( bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['n_events']/bkg_norm_dict_reco['totalEvents'+'_'+selLike[0]][year]*sig_norm_dict_reco['ttbar_'+selLike[0]][year]['n_events']*(1.-bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['norm']) )

            #bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['keep_events_nEvtXS'] =  int( bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['n_events']/bkg_norm_dict_gen['totalEvents'+'_'+selLike[0]][year]*sig_norm_dict_gen['ttbar_'+selLike[0]][year]['n_events']*(1.-bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['norm']) )

            #bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['keep_events_'+selLike[0]] = int(bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['norm']*sig_norm_dict_reco['ttbar_'+selLike[0]][year]['n_events'])
            #bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['keep_events_'+selLike[0]] = int(bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['norm']*sig_norm_dict_gen['ttbar_'+selLike[0]][year]['n_events'])

pprint(bkg_norm_dict_gen)
pprint(bkg_norm_dict_reco)

            

233856.0 263472.0
9696588.0 8859904.0
0.024117349319162575 0.029737568262590656
{'QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8_WSel': {'2016': {'keep_events_nEvt': 7652,
                                                                             'n_events': 615093},
                                                                    '2016_preVFP': {'keep_events_nEvt': 6716,
                                                                                    'n_events': 641281},
                                                                    '2017': {'keep_events_nEvt': 17259,
                                                                             'n_events': 1369305},
                                                                    '2018': {'keep_events_nEvt': 23923,
                                                                             'n_events': 1940774}},
 'QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8_topSel': {'2016': {'keep_events_nEvt'

In [11]:
for selLike in [['WSel','Wlike'],['topSel','toplike']]:

    for year in year_list:
        reco_temp=dict()
        gen_temp=dict()
        cr=0
        cg=0
        for sn in t3_samples_dijets.keys():

            if not(sn.startswith('QCD_HT')): continue
                
            pref = sn.split('_Tune')[0]#'_H7' if ('herwig' in sn.lower()) else ''
            h5_out_path = outputDir+sel
            
            try:
                freco = h5py.File(h5_out_path+f'/{selLike[1]}/{pref}_{selLike[0]}_{year}_reco.h5','r')
                fgen = h5py.File(h5_out_path+f'/{selLike[1]}/{pref}_{selLike[0]}_{year}_gen.h5','r')

                reco_lim = bkg_norm_dict_reco[sn+'_'+selLike[0]][year]['keep_events_nEvt']
                gen_lim = bkg_norm_dict_gen[sn+'_'+selLike[0]][year]['keep_events_nEvt']
                
                print(year,sn,reco_lim,gen_lim)
                
                if not(reco_lim==0):
                    if cr==0:
                        reco_temp['inputs']=freco['inputs'][0:reco_lim]
                        reco_temp['genWeights']=freco['genWeights'][0:reco_lim]
                        reco_temp['recoWeights']=freco['recoWeights'][0:reco_lim]
                        reco_temp['MCScale']=freco['MCScale'][0:reco_lim]
                        reco_temp['target']=freco['target'][0:reco_lim]
                        cr=cr+1
                    else:
                        reco_temp['inputs']=np.concatenate((reco_temp['inputs'],freco['inputs'][0:reco_lim]))
                        reco_temp['genWeights']=np.concatenate((reco_temp['genWeights'],freco['genWeights'][0:reco_lim]))
                        reco_temp['recoWeights']=np.concatenate((reco_temp['recoWeights'],freco['recoWeights'][0:reco_lim]))
                        reco_temp['MCScale']=np.concatenate((reco_temp['MCScale'],freco['MCScale'][0:reco_lim]))
                        reco_temp['target']=np.concatenate((reco_temp['target'],freco['target'][0:reco_lim]))
                        cr=cr+1
                        
                if not(gen_lim==0):
  
                    if cg==0:
                        gen_temp['inputs']=fgen['inputs'][0:gen_lim]
                        gen_temp['genWeights']=fgen['genWeights'][0:gen_lim]
                        gen_temp['MCScale']=fgen['MCScale'][0:gen_lim]
                        gen_temp['target']=fgen['target'][0:gen_lim]
                        cg=cg+1
                    else:
                        gen_temp['inputs']=np.concatenate((gen_temp['inputs'],fgen['inputs'][0:gen_lim]))
                        gen_temp['genWeights']=np.concatenate((gen_temp['genWeights'],fgen['genWeights'][0:gen_lim]))
                        gen_temp['MCScale']=np.concatenate((gen_temp['MCScale'],fgen['MCScale'][0:gen_lim]))
                        gen_temp['target']=np.concatenate((gen_temp['target'],fgen['target'][0:gen_lim]))
                        cg=cg+1
                    
                freco.close()
                fgen.close()
            except OSError:
                print(year,sn,0,0)

        with h5py.File(h5_out_path+f'/{selLike[1]}/QCDHT_{selLike[0]}_{year}_reco_reduced.h5','w') as f:
            f.create_dataset('inputs',data=reco_temp['inputs'])                   
            f.create_dataset('genWeights',data=reco_temp['genWeights'])
            f.create_dataset('recoWeights',data=reco_temp['recoWeights'])
            f.create_dataset('MCScale',data=reco_temp['MCScale'])
            f.create_dataset('target',data=reco_temp['target'])
            f.close()

        with h5py.File(h5_out_path+f'/{selLike[1]}/QCDHT_{selLike[0]}_{year}_gen_reduced.h5','w') as f:
            f.create_dataset('inputs',data=gen_temp['inputs'])
            f.create_dataset('genWeights',data=gen_temp['genWeights'])
            f.create_dataset('MCScale',data=gen_temp['MCScale'])
            f.create_dataset('target',data=gen_temp['target'])
            f.close()
            

2017 QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8 0 0
2017 QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8 0 0
2017 QCD_HT300to500_TuneCP5_PSWeights_13TeV-madgraph-pythia8 2270 2180
2017 QCD_HT500to700_TuneCP5_PSWeights_13TeV-madgraph-pythia8 26271 31968
2017 QCD_HT700to1000_TuneCP5_PSWeights_13TeV-madgraph-pythia8 41198 53840
2017 QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8 12757 17259
2017 QCD_HT1500to2000_TuneCP5_PSWeights_13TeV-madgraph-pythia8 8837 12117
2017 QCD_HT2000toInf_TuneCP5_PSWeights_13TeV-madgraph-pythia8 4333 5973
2018 QCD_HT100to200_TuneCP5_PSWeights_13TeV-madgraph-pythia8 0 0
2018 QCD_HT200to300_TuneCP5_PSWeights_13TeV-madgraph-pythia8 0 0
2018 QCD_HT300to500_TuneCP5_PSWeights_13TeV-madgraph-pythia8 2752 3002
2018 QCD_HT500to700_TuneCP5_PSWeights_13TeV-madgraph-pythia8 33736 42483
2018 QCD_HT700to1000_TuneCP5_PSWeights_13TeV-madgraph-pythia8 56948 74895
2018 QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8 17736 23923
2018 QCD

In [13]:
np.random.RandomState(seed=42)
for selLike in [['WSel','Wlike'],['topSel','toplike']]:
    dataset_dict = dict()
    dataset_dict['reco'] = dict()
    dataset_dict['gen'] = dict()
    for y,year in enumerate(year_list):
        dijet_h5_path = outputDir+'dijetSel'
        Wtop_h5_path = outputDir+selLike[0]
        
        reco_sigFile = h5py.File(Wtop_h5_path+f'/ttbar_{selLike[0]}_{year}_reco.h5','r')
        gen_sigFile = h5py.File(Wtop_h5_path+f'/ttbar_{selLike[0]}_{year}_gen.h5','r')
        
        reco_bkgFile = h5py.File(dijet_h5_path+f'/{selLike[1]}/QCDHT_{selLike[0]}_{year}_reco_reduced.h5','r') 
        gen_bkgFile = h5py.File(dijet_h5_path+f'/{selLike[1]}/QCDHT_{selLike[0]}_{year}_gen_reduced.h5','r') 
        
        print(y,len(reco_sigFile['target']),len(gen_sigFile['target']),len(reco_bkgFile['target']),len(gen_bkgFile['target']))
        if y==0:
            dataset_dict['reco']['inputs'] = np.concatenate((reco_sigFile['inputs'],reco_bkgFile['inputs']))
            dataset_dict['reco']['genWeights'] = np.concatenate((reco_sigFile['genWeights'],reco_bkgFile['genWeights']))
            dataset_dict['reco']['recoWeights'] = np.concatenate((reco_sigFile['recoWeights'],reco_bkgFile['recoWeights']))
            dataset_dict['reco']['MCScale'] = np.concatenate((reco_sigFile['MCScale'],reco_bkgFile['MCScale']))
            dataset_dict['reco']['target'] = np.concatenate((reco_sigFile['target'],reco_bkgFile['target']))

            dataset_dict['gen']['inputs'] = np.concatenate((gen_sigFile['inputs'],gen_bkgFile['inputs']))
            dataset_dict['gen']['genWeights'] = np.concatenate((gen_sigFile['genWeights'],gen_bkgFile['genWeights']))
            dataset_dict['gen']['MCScale'] = np.concatenate((gen_sigFile['MCScale'],gen_bkgFile['MCScale']))
            dataset_dict['gen']['target'] = np.concatenate((gen_sigFile['target'],gen_bkgFile['target']))
        else:
            dataset_dict['reco']['inputs'] = np.concatenate((dataset_dict['reco']['inputs'],reco_bkgFile['inputs']))
            dataset_dict['reco']['genWeights'] = np.concatenate((dataset_dict['reco']['genWeights'],reco_bkgFile['genWeights']))
            dataset_dict['reco']['recoWeights'] = np.concatenate((dataset_dict['reco']['recoWeights'],reco_bkgFile['recoWeights']))
            dataset_dict['reco']['MCScale'] = np.concatenate((dataset_dict['reco']['MCScale'],reco_bkgFile['MCScale']))
            dataset_dict['reco']['target'] = np.concatenate((dataset_dict['reco']['target'],reco_bkgFile['target']))

            dataset_dict['gen']['inputs'] = np.concatenate((dataset_dict['gen']['inputs'],gen_bkgFile['inputs']))
            dataset_dict['gen']['genWeights'] = np.concatenate((dataset_dict['gen']['genWeights'],gen_bkgFile['genWeights']))
            dataset_dict['gen']['MCScale'] = np.concatenate((dataset_dict['gen']['MCScale'],gen_bkgFile['MCScale']))
            dataset_dict['gen']['target'] = np.concatenate((dataset_dict['gen']['target'],gen_bkgFile['target']))
            
        reco_sigFile.close()
        gen_sigFile.close()
        reco_bkgFile.close()
        gen_bkgFile.close()
    
    indices_reco = np.random.shuffle(np.arange(dataset_dict['reco']['inputs'].shape[0]))

    dataset_dict['reco']['inputs']=dataset_dict['reco']['inputs'][indices_reco]
    dataset_dict['reco']['genWeights']=dataset_dict['reco']['genWeights'][indices_reco]
    dataset_dict['reco']['recoWeights']=dataset_dict['reco']['recoWeights'][indices_reco]
    dataset_dict['reco']['MCScale']=dataset_dict['reco']['MCScale'][indices_reco]
    dataset_dict['reco']['target']=dataset_dict['reco']['target'][indices_reco]

    indices_gen = np.random.shuffle(np.arange(dataset_dict['gen']['inputs'].shape[0]))

    dataset_dict['gen']['inputs']=dataset_dict['gen']['inputs'][indices_gen]
    dataset_dict['gen']['genWeights']=dataset_dict['gen']['genWeights'][indices_gen]
    dataset_dict['gen']['MCScale']=dataset_dict['gen']['MCScale'][indices_gen]
    dataset_dict['gen']['target']=dataset_dict['gen']['target'][indices_gen]
    
    with h5py.File(f'V9/ML/Wtop_vs_dijet_reco_{selLike[1]}.h5', 'w') as Wtop_vs_dijet_recof:
        Wtop_vs_dijet_recof.create_dataset('inputs',data=dataset_dict['reco']['inputs'])
        Wtop_vs_dijet_recof.create_dataset('genWeights',data=dataset_dict['reco']['genWeights'])
        Wtop_vs_dijet_recof.create_dataset('recoWeights',data=dataset_dict['reco']['recoWeights'])
        Wtop_vs_dijet_recof.create_dataset('MCScale',data=dataset_dict['reco']['MCScale'])
        Wtop_vs_dijet_recof.create_dataset('target',data=dataset_dict['reco']['target'])
        Wtop_vs_dijet_recof.close()

    with h5py.File(f'V9/ML/Wtop_vs_dijet_gen_{selLike[1]}.h5', 'w') as Wtop_vs_dijet_genf:
        Wtop_vs_dijet_genf.create_dataset('inputs',data=dataset_dict['gen']['inputs'])
        Wtop_vs_dijet_genf.create_dataset('genWeights',data=dataset_dict['gen']['genWeights'])
        Wtop_vs_dijet_genf.create_dataset('MCScale',data=dataset_dict['gen']['MCScale'])
        Wtop_vs_dijet_genf.create_dataset('target',data=dataset_dict['gen']['target'])
        Wtop_vs_dijet_genf.close()

0 95670 123340 95666 123337
1 128994 168678 128991 168675
2 32874 46202 32871 46199
3 37498 51073 37494 51070
0 77420 83760 77417 83758
1 100196 113549 100192 113546
2 26604 31627 26602 31625
3 29636 34536 29634 34532


In [ ]:
#del (runHisto)
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")